In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from ordered_set import OrderedSet


path = "drive/My Drive/hb_case/"  
  
events = pd.read_json(path + "events.json")
meta = pd.read_json(path + "meta.json")

In [ ]:
data = []

for index, row in events.iterrows():
    json_row = row['events']
    
    event_column = json_row['event']
    session_column = json_row['sessionid']
    eventtime_column = json_row['eventtime']
    price_column = json_row['price']
    productid_column = json_row['productid']

    data.append([event_column, session_column, eventtime_column, price_column, productid_column])

df_events = pd.DataFrame(data, columns=['event','sessionid','eventtime','price','productid'])
df_events = df_events[df_events['productid'].notnull()]


In [ ]:
data = []

for index, row in meta.iterrows():
    json_row = row['meta']

    productid_column = json_row['productid']
    brand_column = json_row['brand']
    category_column = json_row['category']
    subcategory_column = json_row['subcategory']
    name_column = json_row['name']

    data.append([productid_column, brand_column, category_column, subcategory_column, name_column])

df_meta = pd.DataFrame(data, columns=['productid','brand','category','subcategory','name'])
df_meta = df_meta[df_meta['productid'].notnull()]

In [ ]:
df_meta.to_csv(path + 'df_meta.csv', index=False)
df_events.to_csv(path + 'df_events.csv', index=False)

In [107]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from ordered_set import OrderedSet
from itertools import combinations
from itertools import permutations
import itertools
!pip install ordered-set

path = "drive/My Drive/hb_case/"  

df_event = pd.read_csv(path + "df_events.csv")
df_meta = pd.read_csv(path + "df_meta.csv")

In [75]:
pd.set_option('display.max_colwidth', None)


def is_column_unique(column):
  return column.unique().shape[0] == column.shape[0]


def join_frames(df_01, df_02, join_colum_list, join_type, report=True):

  join_res = pd.merge(df_01, df_02, how=join_type, on=join_colum_list)

  if report:
    print("frame 01 row size : " + str(df_01.shape[0]))
    print("frame 02 row size : " + str(df_02.shape[0]))
    print("join row size     : " + str(join_res.shape[0]))

  return join_res


def union_frames(df_01, df_02):
  return pd.concat([df_01, df_02])


def get_product_name(df_meta, item_list):
  return df_meta[df_meta['productid'].isin(item_list)]


def get_permutations(a_list):
  lst = list(itertools.permutations(a_list, 2))
  return lst


def get_lift(productid, secondary_productid):
  return df_lift[(df_lift['productid'] == productid) & (df_lift['secondary_productid'] == secondary_productid)]

In [ ]:
df_meta.head()

In [ ]:
df_event.head()

In [108]:
df_event.shape

(387650, 5)

In [109]:

df_event = pd.read_csv(path + "df_events.csv")
df_event_gropped = df_event.groupby('sessionid')['productid'].apply(list).reset_index(name='productid')
df_event_gropped = df_event_gropped[df_event_gropped['productid'].str.len() > 1] 
df_event_groupped_test = df_event_gropped.sample(n = int(df_event_gropped.shape[0] * 0.1))

test_session_id_list = list(df_event_groupped_test['sessionid'].tolist())
df_event = df_event[~df_event['sessionid'].isin(test_session_id_list)]

df_event.shape

(351106, 5)

In [110]:
df_event_groupped_test.shape

(3848, 2)

**ÜRÜN FİYATLARINI ALMA**


In [77]:
# getting items prices

df_product_price = df_event.groupby('productid')['price'].apply(list).reset_index(name='price')
df_product_price['price'] = df_product_price['price'].apply(lambda x: x[0])

df_product_detail = join_frames(df_product_price, df_meta, ['productid'], 'inner')

df_product_detail.head()


frame 01 row size : 10086
frame 02 row size : 10235
join row size     : 10086


,productid,price,brand,category,subcategory,name
0,AILEBIZIZSMTLDGY54,59.99,Barbie,Oyuncak ve Kırtasiye,Oyuncak,Barbie Büyüleyici Parti Bebekleri
1,AILEBIZIZSMTLDHB18,64.99,Barbie,Oyuncak ve Kırtasiye,Oyuncak,Barbie Kariyer Bebekleri
2,AILEBS179526,169.99,Disney Cars,Oyuncak ve Kırtasiye,Oyuncak,Cars Renk Değiştiren Araçlar Mac Tır CKD34
3,AILEBSHSB22037,45.90,Play-Doh,Oyuncak ve Kırtasiye,Oyuncak,Play Doh Bonbon Partı Setı
4,AILEDALIN275101,9.90,Dalin,Bebek,Bebek Bakım ve Sağlığı,Dalin Şampuan Klasik 200ML


**SUPPORT HESAPLAMA**

In [ ]:
# CALCULATE SUPPORT

df_product_support = df_event.groupby(['productid']).size().reset_index(name='counts')
df_product_support['support'] = df_product_support['counts'] / df_product_support.shape[0]

df_product_support.head(10)

**CONFIDENCE HESAPLAMA**

In [ ]:
# ürünler session'lara göre gruplanıp, ürünlerin unique halleri bulunuyor

df_event.sort_values(by=['sessionid', 'productid'], ascending=True, inplace=True)
df_associated_product = df_event.groupby('sessionid')['productid'].apply(list).reset_index(name='productid')

# aynı session'da birden fazla satılan ürünleri tek olarak düşün, sadece diğer(ler)iyle beraber satılmış mı ona bakılacak
df_associated_product['productid_unique'] = df_associated_product['productid'].apply(lambda x: list(OrderedSet(x))) 

# bir session'da tek bir ürünün satıldığı satırları dikkate alma
df_associated_product = df_associated_product[df_associated_product['productid_unique'].str.len() > 1] 

df_associated_product.head(10)

In [ ]:
# aynı session'da bulunmuş olan ürünlerin ikili permütasyonları bulunuyor

df_associated_product.drop(['sessionid', 'productid'], axis=1, inplace=True)
df_associated_product['productid_unique__perm'] = df_associated_product['productid_unique'].apply(lambda x: get_permutations(x)) # aynı session'da birden fazla satılan ürünleri tek olarak düşün, sadece diğer(ler)iyle beraber satılmış mı ona bakılacak

df_associated_product.head(5)

In [ ]:
# tüm ikililer açılıyor

df_associated_product = df_associated_product.explode('productid_unique__perm').reset_index(drop=True) 
df_associated_product.drop(['productid_unique'], axis=1, inplace=True)

df_associated_product.head(10)

In [ ]:
# tüm ikililelerin sayılaıar (combination kullanılmadığı için tüm ürünler için 2 çift geliyor)

df_pair_count = df_associated_product.groupby(['productid_unique__perm']).size().reset_index(name='associated_count')
df_pair_count = df_pair_count[df_pair_count['associated_count'] > 1]
df_pair_count.sort_values(by=['associated_count'], ascending=False, inplace=True)

df_pair_count.head(10)

In [ ]:
# ikililerin düzenlenmiş hali 

df_pair_count[['productid', 'secondary_productid']] = pd.DataFrame(df_pair_count['productid_unique__perm'].tolist(), index=df_pair_count.index)
df_pair_count.drop(['productid_unique__perm'], axis=1, inplace=True)

df_pair_count.head(10)

In [ ]:
# tüm ikililer arasındaki 'confidence' değeri hesaplanıyor

df_confidence = join_frames(df_pair_count, df_product_support, ['productid'], 'inner', False)

df_confidence.head(10)

df_confidence['confidence'] = df_confidence['associated_count'] / df_confidence['counts']

df_confidence.sort_values(by=['confidence'], ascending=False, inplace=True)
df_confidence = df_confidence.rename(columns = {'counts':'main_count'})

df_pre_lift = df_confidence.copy()

df_confidence = df_confidence[['productid', 'secondary_productid', 'main_count', 'associated_count', 'confidence']]

df_confidence.head(50)

**CALCULATE LIFT**

In [ ]:
# lift hesaplanıyor

df_pre_lift = df_pre_lift.rename(columns = {'support': 'main_support'})
df_lift = pd.merge(df_pre_lift, df_product_support, how='inner', left_on='secondary_productid', right_on='productid')
df_lift.drop(['productid_y'], axis=1, inplace=True)
df_lift = df_lift.rename(columns = {'productid_x': 'productid'})

df_lift = df_lift.rename(columns = {'support': 'secondary_support'})
df_lift = df_lift.rename(columns = {'counts': 'secondary_count'})

df_lift = df_lift[['productid', 'secondary_productid', 'main_count', 'secondary_count', 'associated_count', 'main_support', 'secondary_support', 'confidence']]
df_lift['lift'] = df_lift['confidence'] / df_lift['secondary_support']
df_lift.sort_values(by=['lift'], ascending=False, inplace=True)

# lift <= 1 ise iki ürün tamamen alakasız, o değerler filtreleniyor 
df_lift = df_lift[(df_lift['associated_count'] >= 5) & (df_lift['lift'] > 3)]

df_lift.head(50) 


In [ ]:
# NORMALIZE LIFT

df_lift['log_lift'] = np.log(np.log(np.log(df_lift['lift'])))
df_lift['normalized_lift'] = (df_lift['log_lift']-df_lift['log_lift'].min())/(df_lift['log_lift'].max()-df_lift['log_lift'].min())

df_lift.head(50)

In [ ]:
# bulunan lift ile, ürünler ve beraber satıldığı tüm ürünler gruplanıyor

df_apriori = df_lift[['productid', 'secondary_productid', 'normalized_lift']]
df_apriori['suggestion'] = df_lift['secondary_productid'] + " - " + df_lift['normalized_lift'].astype(str)
df_apriori = df_apriori.groupby('productid')['suggestion'].apply(list).reset_index(name='suggestion')

df_apriori.head(50)

In [ ]:
# merak edilen ürünler için deneme yapılabilir :)
item_list = ['HBV00000DJ9Q7', 'HBV00000DJ9PJ']

get_product_name(df_meta, item_list)
get_lift(item_list[0], item_list[1])


**EMBEDDING PREPROCESS**

In [ ]:
df_event = df_event[df_event['productid'].notnull()]

df_event.sort_values(by=['sessionid', 'eventtime'], ascending=True, inplace=True)

df_associated_product = df_event.groupby('sessionid')['productid'].apply(list).reset_index(name='productid')
df_associated_product['temp_column'] = 1
df_associated_product['productid_unique'] = df_associated_product['productid'].apply(lambda x: list(OrderedSet(x)))
df_associated_product = df_associated_product[df_associated_product['productid_unique'].str.len() > 1]
df_associated_product = df_associated_product.groupby('temp_column')['productid_unique'].apply(list).reset_index(name='productid_unique')

word_tokenized_corpus = df_associated_product['productid_unique'].tolist()[0]

word_tokenized_corpus


In [ ]:
!pip install fasttext
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec

embedding_size = 60
window_size = 40
min_word = 5
down_sampling = 1e-2

ft_model_v1 = FastText(word_tokenized_corpus, size=embedding_size, window=window_size, min_count=min_word, sample=down_sampling, sg=1, iter=100)

In [ ]:
from gensim.test.utils import get_tmpfile

ft_model_v1.save(path + 'ft_model_v1.model')

In [ ]:
!pip install fasttext
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec

ft_model_v1 = FastText.load(path + 'ft_model_v1.model')

In [89]:
def apriori_to_reco(productid):
  reco_01_list = df_apriori[df_apriori['productid'] == productid]['suggestion'].tolist()
  if not reco_01_list:
    return []
  
  return reco_01_list[0][:20]


def embedding_to_reco(productid):
  
  similar_word_list = ft_model_v1.wv.similar_by_word(productid, 20)
  similar_word_list = filter(lambda x: x[1] >= 0.75, similar_word_list)

  formatted_list = [" - ".join(map(str, words)) for words in similar_word_list] 
  return formatted_list


def reco_list_to_names(productid, product_similarity_list, source):

  id_list = []
  similarity_list = []
  
  for similarity_pair in product_similarity_list:
    similarity_pair_list = similarity_pair.split(" - ")
    id_list.append(similarity_pair_list[0])
    similarity_list.append(similarity_pair_list[1])

  df = pd.DataFrame(
      {'productid': id_list,
      'similarity': similarity_list,
       'source': source
      })

  return list(df_meta[df_meta['productid'] == productid]['name'])[0], join_frames(df, df_meta, ['productid'], 'inner', False)[['productid', 'name', 'category', 'subcategory', 'similarity', 'source']]


def fill_recommendation_list(df_recommendation, number_of_random_products):

  ignore_list = df_recommendation['productid'].tolist()
  ignore_list.append(product_id)

  product_detail = df_product_detail[df_product_detail['productid'] == product_id]

  price = product_detail['price'].tolist()[0]
  brand = product_detail['brand'].tolist()[0]
  category = product_detail['category'].tolist()[0]
  subcategory = product_detail['subcategory'].tolist()[0]

  target_price_up = price * 3.0
  target_price_down = price * 0.5

  df_filtered_product = df_product_detail[(df_product_detail['subcategory'] == subcategory) & (df_product_detail['price'] >= target_price_down) & (df_product_detail['price'] <= target_price_up)]
  df_filtered_product = df_filtered_product[~df_filtered_product['productid'].isin(ignore_list)]

  df_filtered_product['price_similarity'] = (df_filtered_product['price'] - price).abs()
  df_filtered_product['price_similarity_normalized'] = 1 - (df_filtered_product['price_similarity']-df_filtered_product['price_similarity'].min())/(df_filtered_product['price_similarity'].max()-df_filtered_product['price_similarity'].min()) 

  df_filtered_product['similarity'] = df_filtered_product['price_similarity_normalized'] / 100
  df_filtered_product['source'] = 'random'
  df_filtered_product = df_filtered_product[['productid', 'name', 'category', 'subcategory', 'similarity', 'source']]

  df_similarity_03 = df_filtered_product.sample(number_of_random_products)
  df_similarity_03.sort_values(by=['similarity'], ascending=False, inplace=True)
  df_recommendation = union_frames(df_recommendation, df_similarity_03)

  return df_recommendation


In [ ]:
# fastText'ten gelen örnek tavsiyeler

product_id = 'HBV00000U273W'

reco_list_01 = embedding_to_reco(product_id) 
product, df_similarity_01 = reco_list_to_names(product_id, reco_list_01, 'fastText')
print(product)
print()
df_similarity_01.head(30)

In [ ]:
# apriori'den gelen örnek tavsiyeler

product_id = 'HBV00000U273W'
reco_list_02 = apriori_to_reco(product_id)
product, df_similarity_02 = reco_list_to_names(product_id, reco_list_02, 'apriori')
print(product)
print()
df_similarity_02.head(10) 

In [90]:
def recommend_product(product_id):

  reco_list_01 = embedding_to_reco(product_id) 
  product, df_similarity_01 = reco_list_to_names(product_id, reco_list_01, 'fastText')

  reco_list_02 = apriori_to_reco(product_id)
  product, df_similarity_02 = reco_list_to_names(product_id, reco_list_02, 'apriori')

  print("*** " + product + " ***\n")

  df_recommendation = union_frames(df_similarity_01, df_similarity_02)
  df_recommendation['similarity'] = df_recommendation['similarity'].astype(float)

  # embedding & apriori'den aynıları gelebilir diye productid'ye göre gruplama yapıyoruz
  df_recommendation = df_recommendation.groupby('productid').agg({'name': 'min', 'category': 'min', 'subcategory': 'min', 'similarity': 'max', 'source': 'sum'})
  df_recommendation.reset_index(inplace=True)

  df_recommendation.sort_values(by=['similarity'], ascending=False, inplace=True)

  number_of_target_recommendations = 10
  number_of_recommendations = df_recommendation.shape[0]

  if number_of_recommendations < number_of_target_recommendations:
    
    df_recommendation = fill_recommendation_list(df_recommendation, number_of_target_recommendations - number_of_recommendations)

  return df_recommendation#.head(number_of_target_recommendations)

In [ ]:
#product_id_list = ['HBV00000NE0SY', 'HBV00000QU3Z9', 'HBV00000NE25T', 'HBV00000NG8T3', 'AILEELITDIS5288B', 'HBV00000U27LJ']
#product_id_list = ['HBV00000NE0SY']
product_id_list = ['HBV00000NE21L', 'HBV00000NE0SY', 'HBV00000QU3Z9', 'HBV00000NE25T', 'HBV00000NG8T3', 'AILEELITDIS5288B', 'HBV00000U27LJ'] 
#all_recos = pd.DataFrame(columns=['productid', 'name', 'category', 'subcategory', 'similarity', 'source', 'product_number'])

all_recos = None

count = 1
for product_id in product_id_list:

  # sırayla ürünler için tavsiyler geliyor
  rc = recommend_product(product_id)

  rc['product_number'] = str(count)
  rc['order'] = np.arange(rc.shape[0]) + 1

  # gelen tavsiyler bir araya toplanıyor
  all_recos = union_frames(all_recos, rc)

  all_recos = all_recos[~all_recos['productid'].isin(product_id_list)]

  count += 1


all_recos.head(50)

### farlı ürünler için, aynı ürün tavsiye ediliyorsa, onlara öncelik verilecek, bunun sayısını öğrenmek için
### gruplama yapılıyor. product_number kolonunda, öoklu olan ifadeler concat'lanıyor

res = all_recos.groupby('productid').agg({'name': 'min', 'category': 'min', 'subcategory': 'min', 'similarity': 'max', 'source': 'sum', 'product_number': 'sum', 'order': 'sum'})
res.reset_index(inplace=True)

# bu satırdaki ürün kaç farklı ürün için tavsiye edilmiş
res['same_candidate_number'] = res['product_number'].str.len()

# bunlar öncelikli olacağından üst sıraya alınıyor
res.sort_values(by=['same_candidate_number'], ascending=False, inplace=True)

res.head(50)

In [ ]:
# alltaki kısım, çoklu ürün tavsiyesinde, bir ürün birden fazla girdi için tavsiye edildiyse, onu üst sıraya taşıyor

# birden fazla ürün için tavsiye edilmiş ürünler
mutual_reco = res[res['same_candidate_number'] > 1]

# bir kere tavsiye edilmiş ürünler
product_reco = res[res['same_candidate_number'] <= 1]

# bir kere tavsiye edilenler ürün'ün kendi içindeki tavsiye sırası ve ürün numarasına (product_number) göre küçükten büyüğe sıralanıyor
product_reco.sort_values(by=['order', 'product_number'], ascending=True, inplace=True)

# birden fazla tavsiye edilenler en üstte olacak 
final_reco = union_frames(mutual_reco, product_reco)

final_reco = final_reco.head(10)

# birden fazla tavsiye edilen ürünler yukarıda olacak şekilde, tavsiye edilen ürünler benzerlik puanlarına göre sıralanıyor
final_reco.sort_values(by=['same_candidate_number', 'similarity'], ascending=False, inplace=True)

final_reco.head(10)

In [ ]:
# BU KISIM KONTROL AMAÇLI
#### input olarak diyelim ki 7 giriş oldu ama farklı ürünler için aynıları tavsiye edildiğinde öncelikli olduğundan onlar üste çıkınca
#### diğerlerine sıra gelmeyebiliyor. Bunun için, adına tavsiye yapılmamış ürünler tespit ediliyor (1)
#### dahil edilmeyen ürün kadar satır sondan siliniyor (2)
#### diğer ürünlerin ilk sıradaki tavsiyesi, sırayla sonra ekleniyor

number_of_input = len(product_id_list)
product_index_list = final_reco['product_number'].unique()

product_index_str = '-'.join(product_index_list)

not_included_list = []

# (1)
for i in range(1, number_of_input + 1):
  index = str(i)
  if index not in product_index_str:
    not_included_list.append(index)  


if len(not_included_list) > 0:

  # (2)
  final_reco = final_reco.iloc[:-len(not_included_list)]

  # (3)
  for product_index in not_included_list:
    final_reco = union_frames(final_reco, product_reco[(product_reco['product_number'] == product_index) & (product_reco['order'] == 1)]) 

final_reco.head(10)

In [66]:
def get_final_recommendation(product_id_list):

  all_recos = None

  count = 1
  for pid in product_id_list:

    # sırayla ürünler için tavsiyler geliyor
    rc = recommend_product(pid)
    rc['product_number'] = str(count)
    rc['order'] = np.arange(rc.shape[0]) + 1

    # gelen tavsiyler bir araya toplanıyor
    all_recos = union_frames(all_recos, rc)

    all_recos = all_recos[~all_recos['productid'].isin(product_id_list)]

    count += 1

  all_recos.head(50)

  ### farlı ürünler için, aynı ürün tavsiye ediliyorsa, onlara öncelik verilecek, bunun sayısını öğrenmek için
  ### gruplama yapılıyor. product_number kolonunda, öoklu olan ifadeler concat'lanıyor

  res = all_recos.groupby('productid').agg({'name': 'min', 'category': 'min', 'subcategory': 'min', 'similarity': 'max', 'source': 'sum', 'product_number': 'sum', 'order': 'sum'})
  res.reset_index(inplace=True)

  # bu satırdaki ürün kaç farklı ürün için tavsiye edilmiş
  res['same_candidate_number'] = res['product_number'].str.len()

  # bunlar öncelikli olacağından üst sıraya alınıyor
  res.sort_values(by=['same_candidate_number'], ascending=False, inplace=True)

  mutual_reco = res[res['same_candidate_number'] > 1]

  # bir kere tavsiye edilmiş ürünler
  product_reco = res[res['same_candidate_number'] <= 1]

  # bir kere tavsiye edilenler ürün'ün kendi içindeki tavsiye sırası ve ürün numarasına (product_number) göre küçükten büyüğe sıralanıyor
  product_reco.sort_values(by=['order', 'product_number'], ascending=True, inplace=True)

  # birden fazla tavsiye edilenler en üstte olacak 
  final_reco = union_frames(mutual_reco, product_reco)

  final_reco = final_reco.head(10)

  # birden fazla tavsiye edilen ürünler yukarıda olacak şekilde, tavsiye edilen ürünler benzerlik puanlarına göre sıralanıyor
  final_reco.sort_values(by=['same_candidate_number', 'similarity'], ascending=False, inplace=True)


  # BU KISIM KONTROL AMAÇLI
  #### input olarak diyelim ki 7 giriş oldu ama farklı ürünler için aynıları tavsiye edildiğinde öncelikli olduğundan onlar üste çıkınca
  #### diğerlerine sıra gelmeyebiliyor. Bunun için, adına tavsiye yapılmamış ürünler tespit ediliyor (1)
  #### dahil edilmeyen ürün kadar satır sondan siliniyor (2)
  #### diğer ürünlerin ilk sıradaki tavsiyesi, sırayla sonra ekleniyor

  number_of_input = len(product_id_list)
  product_index_list = final_reco['product_number'].unique()

  product_index_str = '-'.join(product_index_list)

  not_included_list = []

  # (1)
  for i in range(1, number_of_input + 1):
    index = str(i)
    if index not in product_index_str:
      not_included_list.append(index)  


  if len(not_included_list) > 0:

    # (2)
    final_reco = final_reco.iloc[:-len(not_included_list)]

    # (3)
    for product_index in not_included_list:
      final_reco = union_frames(final_reco, product_reco[(product_reco['product_number'] == product_index) & (product_reco['order'] == 1)]) 

  return final_reco.head(10)


In [ ]:
#product_id_list = ['HBV00000NE0SY', 'HBV00000QU3Z9', 'HBV00000NE25T', 'HBV00000NG8T3', 'AILEELITDIS5288B', 'HBV00000U27LJ']
#product_id_list = ['HBV00000NE0SY']
product_id_list = ['HBV00000NE21L', 'HBV00000NE0SY', 'HBV00000QU3Z9', 'HBV00000NE25T', 'HBV00000NG8T3', 'AILEELITDIS5288B', 'HBV00000U27LJ'] 

get_final_recommendation(product_id_list)

**TEST**

In [ ]:
product_id_list = ['HBV00000NE21L', 'HBV00000NE0SY', 'HBV00000QU3Z9', 'HBV00000NE25T', 'HBV00000NG8T3', 'AILEELITDIS5288B', 'HBV00000U27LJ'] 

rs = get_final_recommendation(product_id_list)

global count
count = 0
def get_reco_list(item):
  
  global count
  count += 1
  print(count)

  return list(get_final_recommendation(item)['productid'].tolist())


df_event_groupped_test['recommendation_list'] = df_event_groupped_test['productid'].apply(lambda x: get_reco_list(x[:3])) 


df_event_groupped_test.to_csv(path + 'df_event_groupped_test.csv', index=False)

df_event_groupped_test.shape


In [ ]:
def intersection(lst1, lst2):
  return list(set(lst1) & set(lst2))


df_event_groupped_test['number_of_intersection'] = df_event_groupped_test.apply(lambda x: intersection(x['productid'], x['recommendation_list']), axis=1)
df_filtered_res = df_event_groupped_test[df_event_groupped_test['number_of_intersection'].str.len() > 0] 

print(df_event_groupped_test.shape)
print(df_filtered_res.shape)